<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-4-Sprint-2-NLP/blob/master/module1-text-data/LS_DS_421_Text_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) String Basics

## 1.1 Count Characters

Write a function to count the character frequency of any string, and return the counts as a dictionary.

"google.com" -> {'o': 3, 'g': 2, '.': 1, 'e': 1, 'l': 1, 'm': 1, 'c': 1}

In [0]:
def character_frequency(input):
  
  output = {}
  
  for ch in input:
    if ch in output:
      count = output[ch]
      count += 1
      output[ch] = count
    else:
      output[ch] = 1

  return output

In [2]:
print(f"character frequency in google.com is "\
      f"{character_frequency('google.com')}")

print(f"character frequency in Shreyas is "\
      f"{character_frequency('Shreyas')}")

character frequency in google.com is {'g': 2, 'o': 3, 'l': 1, 'e': 1, '.': 1, 'c': 1, 'm': 1}
character frequency in Shreyas is {'S': 1, 'h': 1, 'r': 1, 'e': 1, 'y': 1, 'a': 1, 's': 1}


## 1.2 Replace Characters

Write a function that takes in two parameters: 1) a string of text and 2) individual character. This function should return a new string of text where any of the specified character has been replaced with dollar signs: `$` 

In [0]:
def replace_characters(input_string, replace_char):
  return input_string.replace(replace_char, '$')

In [4]:
print(f"replace o with $ in google.com is "\
      f"{replace_characters('google.com', 'o')}")

replace o with $ in google.com is g$$gle.c$m


## 1.3 Slice Suffixes

Write a function that removes common suffixes from the ends of words. As a minimum this function should remove the suffixes: "ed",  "ing", "es", "tion", "ly"

"The daring fox leaped through the air gracefully, his eyes fixated on the capitulation of his prey." -> 

"The dar fox leap through the air graceful, his ey fixat on the capitula of his prey."

In [0]:
def slice_suffixes(input_string):
  suffix_list = ["ed", "ing", "es", "tion", "ly"]
  output = ""
  word_list = input_string.split()
  #print(word_list)
  
  for word in word_list:
    #print(f'word {word}')
    
    split_word = word.partition(',')
    partition = ""
    if len(split_word) != 1:
      word = split_word[0]
      partition = split_word[1]
      
    for suffix in suffix_list:
      #print(f'suffix {suffix}')
      
      suffix_len = len(suffix)
      if word[-suffix_len:] == suffix:
        #print(f'copying {word[:suffix_len]} to output')
        if len(partition) != 0:
          output += word[:-suffix_len] + partition + " "
        else:
          output += word[:-suffix_len] + " "
        
        break
        
      if suffix == 'ly':
        #print(f'copying {word} to output')
        if len(partition) != 0:
          output += word + partition + " "
        else:
          output += word + " "
    
  return output.strip()

In [6]:
input_string = "The daring fox leaped through the air gracefully, his eyes fixated on the capitulation of his prey."
print(input_string)
print(slice_suffixes(input_string))

The daring fox leaped through the air gracefully, his eyes fixated on the capitulation of his prey.
The dar fox leap through the air graceful, his ey fixat on the capitula of his prey.


## 1.4 Remove Stopwords

Write a function that removes common "stopwords" from text. 

In [0]:
stopwords = ['i','me','my','myself','we''our','ours','ourselves',
'you','your','yours','yourself','yourselves','he','him','his','himself',
'she','her','hers','herself','it','its','itself','they','them','their',
'theirs','themselves','what','which','who','whom','this','that','these',
'those','am','is','are','was','were','be','been','being','have','has',
'had','having','do','does','did','doing','a','an','the','and','but',
'if','or','because','as','until','while','of','at','by','for','with',
'about','against','between','into','through','during','before','after',
'above','below','to','from','up','down','in','out','on','off','over',
'under','again','further','then','once','here','there','when','where',
'why','how','all','any','both','each','few','more','most','other','some',
'such','no','nor','not','only','own','same','so','than','too','very',
's','t','can','will','just','don','should','now']


def remove_stopwords(input_string):
  word_list = input_string.split()
  output = ""
  
  for word in word_list:
    if word.lower() not in stopwords:
      output += word + " "
    #else:
      #print(word)
  
  return output

In [8]:
print(f"Input String:\n{input_string}")
no_stopwords = remove_stopwords(input_string)
print(f"Removed Stopwords\n{no_stopwords}")
print(f"Removed Suffixes\n{slice_suffixes(no_stopwords)}")

Input String:
The daring fox leaped through the air gracefully, his eyes fixated on the capitulation of his prey.
Removed Stopwords
daring fox leaped air gracefully, eyes fixated capitulation prey. 
Removed Suffixes
dar fox leap air graceful, ey fixat capitula prey.


## 1.5 Vectorize Words

Below is a list of three strings. Each string is a job listing with the job title of "data scientist" from indeed.com. Write a function that does two things:

1) Removes stopwords from each listing (uses above function)

2) Creates a dataframe where the header of each column is a particular word and each cell of the dataframe should be a 1 or 0 denoting whether that word is present or not in the job listing a body of text. The final dataframe should only have 3 rows, one for each of the three job listings.

Your final dataframe should not include any of the stopwords.

In [0]:
job_listings = ['Part-time, Contract, Internship\nSr. Machine Learning/Data Scientist\n\ndata245, Bannockburn, IL seeks data scientists.\n\nWe are open to all levels of experience (down to an intern) as we are building a team around new initiatives.\n\nYou will be developing state of the art algorithms to power various aspects of highly complex business models\nYou can articulate and understand a business problem, identify challenges, formulate the machine learning problem or NLP problems and provide/prototype solutions\nYou will provide technical leadership, identify and understand key business challenges and opportunities, and develop end-to-end software solutions using machine learning/NLP and optimization methods.\n\nYou will collaborate extensively with internal and external partners, program management, and, at a senior level, the engineering team to ensure that solution meet business needs, permit valid inferences and have functional feasibility\nYou will collect and manipulate large volumes of data; build new and improved techniques and/or solutions for data collection, management, and usage\nYou will communicate results in a comprehensible manner to all levels of the company (field teams up to Snr. Management) - this will require client facing in the future - but not initially\nYou will brainstorm with other team members and leadership - who has 30 plus years experience in the industry that requires the solution.\nRequirements:\n\nPHD or MS in Statistics, Machine Learning, or Computer Science (or technical degree with commensurate industry experience)\nIdeally the Senior position will possess at least 3 years of relevant work - or academic academic experience, as a Data Scientist / Machine Learning professional.\nExpertise in NLP a bonus\nStrong algorithmic design skills\nOther positions require less tenure, but the same relevant ML understanding.\nPrevious hands on experience, or thesis dedicated to the same\nDeep understanding of classic machine learning and deep learning theory, and extensive hands-on experience putting it into practice\nExcellent understanding of machine learning algorithms, processes, tools and platforms including CNN, RNN, NLP, tensorflow, keras, etc.\nPython proficiency is must\nApplied experience with machine learning on large datasets\\sparse data with structured and unstructured data.\nExperience with deep learning, and their optimizations for efficient implementation.\nGreat communication skills, ability to explain predictive analytics to non-technical audience (not client facing yet, and no sales)\nExperienced in predictive modelling.\nExecute analytical experiments methodically while outputting reproducible research.\nExcited to change an industry struggling to control costs.\nGood to have – Familiar with one or more programming languages e.g. C++ / Java / Android / iOS".\nJob Types: Full-time, Part-time, Internship, Contract\n\nSalary: $75,000.00 to $125,000.00 /year\n\nEducation:\n\nMaster\'s (Preferred)\nWork authorization:\n\nUnited States (Preferred)\nHours per week:\n\n30-39\nOvertime often available:\n\nNo\nContract Length:\n\nMore than 1 year\nTypical end time:\n\n5PM', "$96,970 - $148,967 a year\nThe professionals at the National Security Agency (NSA) have one common goal: to protect our nation. The mission requires a strong offense and a steadfast defense. The offense collects, processes, and disseminates intelligence information derived from foreign signals for intelligence and counterintelligence purposes. The defense prevents adversaries from gaining access to sensitive classified national security information. NSA is the nation's leader in providing foreign signals intelligence while also protecting U.S. government information systems, forging the frontier of communications, and data analysis. We serve the American people by applying technical skills to meaningful work, keeping our friends and families safe for generations to come. You will make a lasting impact serving your country as a Data Scientist at the National Security Agency, using your curiosity to analyze large data sets to inform decision-making against foreign threats. We are looking for critical thinkers, problem solvers, and motivated individuals who are enthusiastic about data and believe that answers to hard questions lie in the yet-to-be-told story of diverse, complicated data sets. You will employ your mathematical science, computer science, and quantitative analysis skills to ensure solutions to complex data problems and take full advantage of the NSA's software and hardware capabilities in all areas of our enterprise, including analytic capabilities, research, and foreign intelligence operations. Data Scientists are hired into positions directly supporting a technical mission office or the Data Scientist Development Program (DSDP). The NSA/CSS Data Scientist Development Program is a three-year opportunity to build your data science talent, experience the breadth of data science at NSA through six- to nine-month assignments in a variety of diverse organizations, and collaborate with NSA's experts in the field of data science. You will have opportunities to attend technical conferences with experts from industry and academia. You will routinely discuss and share NSA's challenges and successes at weekly technical roundtables. We foster an environment where you will develop your data science skills, allowing you to quickly contribute to NSA's mission. As a member of a technical mission office or the DSDP, Data Scientists tackle challenging real-world problems leveraging big data, high-performance computing, machine learning, and a breadth of other methodologies. As a Data Scientist at NSA, responsibilities may include: - Collecting and combining data from multiple sources - Uncovering and exploring anomalous data (including metadata) - Applying the scientific process to data evaluation, performing statistical inference, and data mining - Developing analytic plans, engineer supporting algorithms, and design and implement solutions which execute analytic plans. - Designing and developing tools and techniques for analysis - Analyzing data using mathematical/statistical methods - Evaluating, documenting, and communicating research processes, analyses, and results to customers, peers, and leadership - Creating interpretable visualizations\n\nSkills\n\nThe ideal candidate is someone with a desire for continual learning and strong problem-solving, analytic and interpersonal skills. You might be a great fit for our team if any of the following describe you: - Completed a degree program in the fields of mathematics, statistics, computer science, computational sciences, or a passion for rigorous analysis of data - Tenacity, integrity, persistence, and willingness to learn - Ability to solve complex problems - Use critical thinking and reasoning to make analytic determinations - Works effectively in a collaborative environment - Strong communications skills to both technical and non-technical audiences - The desire to serve over 300 million fellow Americans and make a difference in world events\n\nPay, Benefits, & Work Schedule\n\nOn the job training, internal NSA courses, and external training will be made available based on the need and experience of the selectee. Monday - Friday, with basic 8 hr/day requirements between 0800 to 1800 (flexible)\n\nPosition Summary\n\nNSA is in search of Computer Science professionals to solve complex problems, test innovative approaches and research new solutions to storing, manipulating, and presenting information. We are looking for you to apply your computer science expertise to projects that seek to create new standards for the transformation of information. If you want to develop technologies and tools and be a part of cutting edge innovations ' join our team of experts! Help protect national security interests as part of the world's most advanced team of computer science professionals!\n\nMandatory Qualification Reqs\n\nCandidates for the NSA's Data Scientist roles are asked to complete a data science examination evaluating their knowledge of statistics, mathematics, and computer science topics that pertain to data science work. Passing this examination is a requirement in order to be considered for selection into a data scientist position. Salary Range: $69,545 - $86,659 (Entry Level/Developmental) *The qualifications listed are the minimum acceptable to be considered for the position. Salary offers are based on candidates' education level and years of experience relevant to the position and also take into account information provided by the hiring manager/organization regarding the work level for the position. Entry is with a Bachelor's degree and no experience. An Associate's degree plus 2 years of relevant experience may be considered for individuals with in-depth experience that is clearly related to the position. Degree must be in Mathematics, Applied Mathematics, Statistics, Applied Statistics, Machine Learning, Data Science, Operations Research, or Computer Science. A degree in a related field (e.g., Computer Information Systems, Engineering), a degree in the physical/hard sciences (e.g. physics, chemistry, biology, astronomy), or other science disciplines (i.e., behavioral, social, and life) may be considered if it includes a concentration of coursework (typically 5 or more courses) in advanced mathematics (typically 200 level or higher; such as calculus, differential equations, discrete mathematics) and/or computer science (e.g., algorithms, programming, data structures, data mining, artificial intelligence). College-level Algebra or other math courses intended to meet a basic college level requirement, or upper level math courses designated as elementary or basic do not count. Note: A broader range of degrees will be considered if accompanied by a Certificate in Data Science from an accredited college/university.\n\nRelevant experience must be in designing/implementing machine learning, data mining, advanced analytical algorithms, programming, data science, advanced statistical analysis, artificial intelligence, and/or software engineering. Experience in more than one area is strongly preferred. Salary Range: $80,445 - $107,140 (Full Performance) *The qualifications listed are the minimum acceptable to be considered for the position. Salary offers are based on candidates' education level and years of experience relevant to the position and also take into account information provided by the hiring manager/organization regarding the work level for the position. Entry is with a Bachelor's degree plus 3 years of relevant experience or a Master's degree plus 1 year of relevant experience or a Doctoral degree and no experience. An Associate's degree plus 5 years of relevant experience may be considered for individuals with in-depth experience that is clearly related to the position.\nDegree must be in Mathematics, Applied Mathematics, Statistics, Applied Statistics, Machine Learning, Data Science, Operations Research, or Computer Science. A degree in a related field (e.g., Computer Information Systems, Engineering), a degree in the physical/hard sciences (e.g. physics, chemistry, biology, astronomy), or other science disciplines (i.e., behavioral, social, and life) may be considered if it includes a concentration of coursework (typically 5 or more courses) in advanced mathematics (typically 200 level or higher; such as calculus, differential equations, discrete mathematics) and/or computer science (e.g., algorithms, programming, data structures, data mining, artificial intelligence). College-level Algebra or other math courses intended to meet a basic college level requirement, or upper level math courses designated as elementary or basic do not count. Note: A broader range of degrees will be considered if accompanied by a Certificate in Data Science from an accredited college/university. Relevant experience must be in two or more of the following: designing/implementing machine learning, data mining, advanced analytical algorithms, programming, data science, advanced statistical analysis, artificial intelligence, or software engineering. Salary Range: $96,970 - $148,967 (Senior) *The qualifications listed are the minimum acceptable to be considered for the position. Salary offers are based on candidates' education level and years of experience relevant to the position and also take into account information provided by the hiring manager/organization regarding the work level for the position. Entry is with a Bachelor's degree plus 6 years of relevant experience or a Master's degree plus 4 years of relevant experience or a Doctoral degree plus 2 years of relevant experience. An Associate's degree plus 8 years of relevant experience may be considered for individuals with in-depth experience that is clearly related to the position. Degree must be in Mathematics, Applied Mathematics, Statistics, Applied Statistics, Machine Learning, Data Science, Operations Research, or Computer Science. A degree in a related field (e.g., Computer Information Systems, Engineering), a degree in the physical/hard sciences (e.g., physics, chemistry, biology, astronomy), or other science disciplines (i.e., behavioral, social, life) may be considered if it includes a concentration of coursework (typically 5 or more courses) in advanced mathematics (typically 200 level or higher; such as calculus, differential equations, discrete mathematics) and/or computer science (e.g., algorithms, programming, data structures, data mining, artificial intelligence). College-level Algebra or other math courses intended to meet a basic college level requirement, or upper level math courses designated as elementary or basic do not count. Note: A broader range of degrees will be considered if accompanied by a Certificate in Data Science from an accredited college/university. Relevant experience must be in two or more of the following: designing/implementing machine learning, data mining, advanced analytical algorithms, programming, data science, advanced statistical analysis, artificial intelligence, or software engineering. Salary Range: $134,789- $164,200 (Expert) *The qualifications listed are the minimum acceptable to be considered for the position. Salary offers are based on candidates' education level and years of experience relevant to the position and also take into account information provided by the hiring manager/organization regarding the work level for the position. Entry is with a Bachelor's degree plus 9 years of relevant experience or a Master's degree plus 7 years of relevant experience or a Doctoral degree plus 5 years of relevant experience. An Associate's degree plus 11 years of relevant experience may be considered for individuals with in-depth experience that is clearly related to the position. Degree must be in Mathematics, Applied Mathematics, Statistics, Applied Statistics, Machine Learning, Data Science, Operations Research, or Computer Science. A degree in a related field (e.g., Computer Information Systems, Engineering), a degree in the physical/hard sciences (e.g., physics, chemistry, biology, astronomy), or other science disciplines (i.e., behavioral, social, life) may be considered if it includes a concentration of coursework (typically 5 or more courses) in advanced mathematics (typically 200 level or higher; such as calculus, differential equations, discrete mathematics) and/or computer science (e.g., algorithms, programming, data structures, data mining, artificial intelligence). College-level Algebra or other math courses intended to meet a basic college level requirement, or upper level math courses designated as elementary or basic do not count. Note: A broader range of degrees will be considered if accompanied by a Certificate in Data Science from an accredited college/university. Relevant experience must be in two or more of the following: designing/implementing machine learning, data mining, advanced analytical algorithms, programming, data science, advanced statistical analysis, artificial intelligence, or software engineering.\n\nHow To Apply - External\n\nTo apply for this position, please click the 'Apply' button located at the top right of this posting. After completing the application for the first time, or reviewing previously entered information, and clicking the 'Submit' button, you will receive a confirmation email. Please ensure your spam filters are configured to accept emails from noreply@intelligencecareers.gov. ***PLEASE NOTE: U.S. Citizenship is required for all applicants. Reasonable accommodations provided to applicants with disabilities during the application and hiring process where appropriate. NSA is an equal opportunity employer and abides by applicable employment laws and regulations. All applicants and employees are subject to random drug testing in accordance with Executive Order 12564. Employment is contingent upon successful completion of a security background investigation and polygraph. This position is a Defense Civilian Intelligence Personnel System (DCIPS) position in the Excepted Service under 10 U.S.C. 1601. DoD Components with DCIPS positions apply Veterans' Preference to eligible candidates as defined by Section 2108 of Title 5 USC, in accordance with the procedures provided in DoD Instruction 1400.25, Volume 2005, DCIPS Employment and Placement. If you are a veteran claiming veterans' preference, as defined by Section 2108 of Title 5 U.S.C., you may be asked to submit documents verifying your eligibility. Please note that you may be asked a series of questions depending on the position you apply for. Your responses will be used as part of the screening process of your application and will assist in determining your eligibility for the position. Be sure to elaborate on experiences in your resume. Failure to provide the required information or providing inaccurate information will result in your application not being considered for this position. Only those applicants who meet the qualifications for the position will be contacted to begin employment processing. Please Note: Job Posting could close earlier than the closing date due to sufficient number of applicants or position no longer available. We encourage you to apply as soon as possible.\n\nDCIPS Disclaimer\n\nThe National Security Agency (NSA) is part of the DoD Intelligence Community Defense Civilian Intelligence Personnel System (DCIPS). All positions in the NSA are in the Excepted Services under 10 United States Codes (USC) 1601 appointment authority.", '\nMinneapolis, Providence or Framingham\n\nWho is Virgin Pulse?\nVirgin Pulse, founded as part of Sir Richard Branson’s famed Virgin Group, helps organizations build employee health and wellbeing into the DNA of their corporate cultures. As the only company to deliver a powerful, mobile-first digital platform infused with live services, including coaching and biometric screenings, Virgin Pulse’s takes a high-tech-meets-high-touch-approach to engage employees in improving across all aspects of their health and wellbeing, every day – from prevention and building a healthy lifestyle to condition and disease management to condition reversal, all while engaging users daily in building and sustaining healthy habits and behaviors. A global leader in health and wellbeing, Virgin Pulse is committed to helping change lives and businesses around the world for good so that people and organizations can thrive, together. Today, more than 3100 organizations across the globe are using Virgin Pulse’s solutions to improve health, employee wellbeing and engagement, reduce costs and create strong workplace cultures.\n\nWho are our employees?\nAt Virgin Pulse we’re passionate about changing lives for good. We want to make a difference in the world by helping people be healthy so they can perform at their best, every day, at work and home. Our award-winning solutions support leading employers in improving and simplifying the employee health and wellbeing journey and engaging people in all aspects of their health. But our world-class products and programs are nothing without our people – the employees who design, build, promote, sell, test and perfect the latest innovations in workplace health and wellbeing. Our people are our top priority and we invest in their health and happiness. At Virgin Pulse, we have so much more than a strong, supportive company culture – have a shared vision for a healthier, happier world.\nWho you are.\nYou are an experienced Data Scientist who is capable of providing support to our organization’s efforts to maintain an innovative leadership position in the employee engagement SaaS industry. The Data Scientist accesses datasets from various sources, conducts analysis, and presents the findings of each analytic and reporting project. The incumbent will be able to interpret the findings and clearly communicate results and recommendations to internal and external customers. Moreover, you are a professional who is self-directed and thrives working in a fast-paced, collaborative environment, in which expectations are high both for the quality and speed of work.\nIn the role of a Data Scientist you will wear many hats and your skills will be crucial in the following:\nWrite SQL, R, Python programs to access, clean, and transform required data prior to analysis and reporting\nConsult to and collaborate with analytics and client reporting team members to ensure appropriate data is analyzed and that results are provided in a format consistent with standard and customized client reporting services\nTroubleshoot and perform data audits to ensure and improve data integrity; investigate and resolve data discrepancies\nPlan and manage data analytic and reporting process to ensure the projects remain on schedule\nConduct ad hoc analysis as required using varied analytical tools and techniques\nSupport Client Success, sales and Marketing staff with direct communication with Virgin Pulse clients and prospects regarding the results of the analysis\nAchieve annual Key Performance Indicator objectives, which can include report volumes and scope, internal and external client satisfaction, introducing new areas of data and analysis, and influencing company product and process decisions\n\nWhat you bring to the team.\nIn order to represent the best of what we have to offer, you come to us with a multitude of positive attributes including:\nA bachelor’s degree in statistics, computer science, economics, or related field; Master’s degree is a plus\nA minimum of four years of work experience in a similar position\nExperience with data and analytic programming languages such as SQL, R, Python\nExperience with data visualization tools and techniques preferred\nExperience with producing and delivering results using varied media (i.e., multiple MS office formats, dashboards/visualization tools, and potentially other formats)\nExperience in employee health management/health engagement industry preferred\n\nIn addition, you possess the following additional competencies and characteristics:\nStrong analytical skills, with an emphasis on quantitative analysis, descriptive and inferential statistics\nExpertise in statistical analytical software, or the ability to learn through prior experience tools such as SAS, Stata, R, SPSS or similar statistical software\nStrong consulting, communication and presentation skills\nAdvanced R, SQL and database programming skills, experience with MS SQL Server, RedShift, Postgres, and Cassandra/NOSQL databases\nExperience working with large-scale datasets and multiple projects simultaneously\nCreative energy, self-starter, works equally well independently and collaboratively']

In [0]:
import pandas as pd

In [0]:
def vectorise_words(input):
  job_records = []
  word_vector = set()
  
  for job in input:
    # Convert the row to lower
    job = job.lower()
    # Remove stop words
    job = remove_stopwords(job)
    # Normalized white spaces
    job = job.split()
    job_records.append(" ".join(job))
    word_vector.update(job)
  
  # Consolidate the result into tuple
  for_dataframe = []
  for job in job_records:
    data_list = []
    for word in word_vector: 
      if word in job:
        data_list.append(1)
      else:
        data_list.append(0)
      
    for_dataframe.append(data_list)
  
  df = pd.DataFrame(for_dataframe, columns=word_vector)
  
  return df

In [12]:
df = vectorise_words(job_listings)
df.shape

(3, 1043)

In [13]:
df.head()

,engineer,team,united,yet-to-be-told,"peers,",recommendations,rigorous,plus,visualizations,"integrity,",...,drug,(flexible),close,available.,analytics,also,"agency,",derived,process,emails
0,1,1,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
2,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


# 2) Regex + Pandas Practice

Load the contents of the following text file into your notebook:
<https://raw.githubusercontent.com/CoreyMSchafer/code_snippets/master/Python-Regular-Expressions/data.txt>

If you're using Google Colab then you will have to download the raw file from github and either upload to your google drive or upload it to the notebook like we have done with CSVs in the past.

## Make sure your notebook has access to the .txt file

In [14]:
from google.colab import files
uploaded = files.upload()

Saving data.txt to data (4).txt


## Read in the contents of the file

In [15]:
with open('data.txt', 'r', encoding='utf-8') as f:
  contents = f.read()
  
contents

"Dave Martin\n615-555-7164\n173 Main St., Springfield RI 55924\ndavemartin@bogusemail.com\n\nCharles Harris\n800-555-5669\n969 High St., Atlantis VA 34075\ncharlesharris@bogusemail.com\n\nEric Williams\n560-555-5153\n806 1st St., Faketown AK 86847\nlaurawilliams@bogusemail.com\n\nCorey Jefferson\n900-555-9340\n826 Elm St., Epicburg NE 10671\ncoreyjefferson@bogusemail.com\n\nJennifer Martin-White\n714-555-7405\n212 Cedar St., Sunnydale CT 74983\njenniferwhite@bogusemail.com\n\nErick Davis\n800-555-6771\n519 Washington St., Olympus TN 32425\ntomdavis@bogusemail.com\n\nNeil Patterson\n783-555-4799\n625 Oak St., Dawnstar IL 61914\nneilpatterson@bogusemail.com\n\nLaura Jefferson\n516-555-4615\n890 Main St., Pythonville LA 29947\nlaurajefferson@bogusemail.com\n\nMaria Johnson\n127-555-1867\n884 High St., Braavos ME 43597\nmariajohnson@bogusemail.com\n\nMichael Arnold\n608-555-4938\n249 Elm St., Quahog OR 90938\nmichaelarnold@bogusemail.com\n\nMichael Smith\n568-555-6051\n619 Park St., Winter

## Turn the "unstructured" .txt file into a "structured" dataframe
Once you have read in the file's contents your task is to get this unstructured text data into a dataframe with the following headers:

- First Name
- Last Name
- Email
- Phone Number
- Street Address
- City
- State
- Zipcode

In [0]:
##### Your Code Here #####
import re

In [0]:
regex_name = r"[a-zA-Z]{3,} [a-zA-Z\-]{3,}" 
name = re.findall(regex_name, contents)

# Deleting non-names which were matching.
name.remove("Vice City")
name.remove("Vice City")
name.remove("South Park")

In [18]:
df = pd.DataFrame(name, columns=['Full Name'])
df['First Name'] = df['Full Name'].apply(lambda x: x.split(" ")[0])
df['Last Name'] = df['Full Name'].apply(lambda x: x.split(" ")[1])
df.drop(columns='Full Name', inplace=True)
df.head()

,First Name,Last Name
0,Dave,Martin
1,Charles,Harris
2,Eric,Williams
3,Corey,Jefferson
4,Jennifer,Martin-White


In [19]:
regex_email = r"([a-z]+@[a-z]+.com)" 
email = re.findall(regex_email, contents)
print(len(email))
#email

100


In [20]:
df['Email'] = email
df.head()

,First Name,Last Name,Email
0,Dave,Martin,davemartin@bogusemail.com
1,Charles,Harris,charlesharris@bogusemail.com
2,Eric,Williams,laurawilliams@bogusemail.com
3,Corey,Jefferson,coreyjefferson@bogusemail.com
4,Jennifer,Martin-White,jenniferwhite@bogusemail.com


In [21]:
regex_phone = r"\d{3}-\d{3}-\d{4}"
phone = re.findall(regex_phone, contents)
print(len(phone))
#phone

100


In [0]:
df['Phone'] = phone

In [23]:
regex_address = r"(\d+ [1a-zA-Z]+ St)., ([a-zA-Z-\s']+) ([A-Z]{2}) (\d+)"
address = re.findall(regex_address, contents)
print(len(address))
#address

100


In [24]:
df['Address'] = address
df['Street Address'] = df['Address'].apply(lambda x: x[0])
df['City'] = df['Address'].apply(lambda x: x[1])
df['State'] = df['Address'].apply(lambda x: x[2])
df['Zipcode'] = df['Address'].apply(lambda x: x[3])
df.drop(columns='Address', inplace=True)
print(df.shape)
df.head()

(100, 8)


,First Name,Last Name,Email,Phone,Street Address,City,State,Zipcode
0,Dave,Martin,davemartin@bogusemail.com,615-555-7164,173 Main St,Springfield,RI,55924
1,Charles,Harris,charlesharris@bogusemail.com,800-555-5669,969 High St,Atlantis,VA,34075
2,Eric,Williams,laurawilliams@bogusemail.com,560-555-5153,806 1st St,Faketown,AK,86847
3,Corey,Jefferson,coreyjefferson@bogusemail.com,900-555-9340,826 Elm St,Epicburg,NE,10671
4,Jennifer,Martin-White,jenniferwhite@bogusemail.com,714-555-7405,212 Cedar St,Sunnydale,CT,74983


# 3) Web Scraping + Regex + Pandas

Scrape the unordered list of information about Ohio University President's salaries from this article: 
[Ohio Private University President's Salaries](https://www.cleveland.com/metro/2017/12/case_western_reserve_university_president_barbara_snyders_base_salary_and_bonus_pay_tops_among_private_colleges_in_ohio.html)

Get the data from this webpage into a dataframe with the following headers:

- First Name
- Last Name
- School
- Salary

Salary information should be stored as an integer and not have "$" or commas ","


In [0]:
##### Your Code Here #####
from bs4 import BeautifulSoup
import requests

In [26]:
page_to_scrape = 'https://www.cleveland.com/metro/2017/12/case_western_reserve_university_president_barbara_snyders_base_salary_and_bonus_pay_tops_among_private_colleges_in_ohio.html'

# This makes a request to the page just as if we were navigating
# to that page in the browser
page = requests.get(page_to_scrape)

# The "soup" is the parsed html of a webpage.
soup = BeautifulSoup(page.text, 'html.parser')
print(soup)

<!DOCTYPE doctype html>
 <html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"> <head> <script>window.serviceCallbacks=[];window.region_redirect_url="";if(window.location.href===window.region_redirect_url&&typeof Storage!=="undefined"){var region=window.localStorage.getItem("region-item");if(region)window.location.replace(window.region_redirect_url+region)};</script> <title>Case Western Reserve University President Barbara Snyder's base salary and bonus pay tops among private colleges in Ohio - cleveland.com</title> <meta content="Case Western Reserve University President Barbara Snyder's $700,312 base pay and $175,000 bonus in 2015 again made her the highest paid private college leader in Ohio, according to an annual report." name="Description"/> <meta charset="utf-8"/> <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/> <meta content="1000003022874

In [27]:
president_list = soup.find(class_='article__unordered-list')
print(president_list)

<ul class="article__unordered-list"> <li class="article__unordered-list-item">Grant Cornwell, College of Wooster (left in 2015): $911,651</li> <li class="article__unordered-list-item">Marvin Krislov, Oberlin College (left in 2016):  $829,913</li> <li class="article__unordered-list-item">Mark Roosevelt, Antioch College, (left in 2015): $507,672</li> <li class="article__unordered-list-item">Laurie Joyner, Wittenberg University (left in 2015): $463,504</li> <li class="article__unordered-list-item">Richard Giese, University of Mount Union (left in 2015): $453,800</li> <li class="article__unordered-list-item">Sean Decatur,Kenyon College: $451,698</li> <li class="article__unordered-list-item">Adam Weinberg, Denison University: $435,322</li> <li class="article__unordered-list-item">Daniel Dibiasio, Ohio Northern University: $414,716</li> <li class="article__unordered-list-item">Denvy Bowman, Capital University (left in 2016): $388,570</li> <li class="article__unordered-list-item">Anne Steele,

In [28]:
president_list_items = president_list.find_all(class_='article__unordered-list-item')
print(president_list_items)

[<li class="article__unordered-list-item">Grant Cornwell, College of Wooster (left in 2015): $911,651</li>, <li class="article__unordered-list-item">Marvin Krislov, Oberlin College (left in 2016):  $829,913</li>, <li class="article__unordered-list-item">Mark Roosevelt, Antioch College, (left in 2015): $507,672</li>, <li class="article__unordered-list-item">Laurie Joyner, Wittenberg University (left in 2015): $463,504</li>, <li class="article__unordered-list-item">Richard Giese, University of Mount Union (left in 2015): $453,800</li>, <li class="article__unordered-list-item">Sean Decatur,Kenyon College: $451,698</li>, <li class="article__unordered-list-item">Adam Weinberg, Denison University: $435,322</li>, <li class="article__unordered-list-item">Daniel Dibiasio, Ohio Northern University: $414,716</li>, <li class="article__unordered-list-item">Denvy Bowman, Capital University (left in 2016): $388,570</li>, <li class="article__unordered-list-item">Anne Steele, Muskingum University (left

In [29]:
for president in president_list_items:
    print(president.text)

Grant Cornwell, College of Wooster (left in 2015): $911,651
Marvin Krislov, Oberlin College (left in 2016):  $829,913
Mark Roosevelt, Antioch College, (left in 2015): $507,672
Laurie Joyner, Wittenberg University (left in 2015): $463,504
Richard Giese, University of Mount Union (left in 2015): $453,800
Sean Decatur,Kenyon College: $451,698
Adam Weinberg, Denison University: $435,322
Daniel Dibiasio, Ohio Northern University: $414,716
Denvy Bowman, Capital University (left in 2016): $388,570
Anne Steele, Muskingum University (left in 2016): $384,233
Kathy Krendl, Otterbein University:  $378,035
Rockwell Jones, Ohio Wesleyan University: $366,625
Robert Helmer, Baldwin Wallace University: $365,616
Robert Huntington, Heidelberg University: $300,005
Lori Varlotta, Hiram College: $293,336
Joseph Bruno, Marietta College (left in 2016): $288,295
W. Richard Merriman Jr., University of Mount Union (started in June 2015): $221,761


In [30]:
firstname = []
lastname = []
college = []
salary = []

for president in president_list_items:
    fullname = president.text.split(",")[0].split(" ")
    firstname.append(" ".join(fullname[:len(fullname)//2]))
    lastname.append(" ".join(fullname[len(fullname)//2:]))
    college.append(president.text.split(",")[1].split(":")[0].strip())
    
    regex_salary = r"(\$\d{3},\d{3})"
    salary.append(re.search(regex_salary, president.text)[0])
    
print(len(firstname))
print(firstname)
print(len(lastname))
print(lastname)
print(len(college))
print(college)
print(len(salary))
print(salary)

17
['Grant', 'Marvin', 'Mark', 'Laurie', 'Richard', 'Sean', 'Adam', 'Daniel', 'Denvy', 'Anne', 'Kathy', 'Rockwell', 'Robert', 'Robert', 'Lori', 'Joseph', 'W. Richard']
17
['Cornwell', 'Krislov', 'Roosevelt', 'Joyner', 'Giese', 'Decatur', 'Weinberg', 'Dibiasio', 'Bowman', 'Steele', 'Krendl', 'Jones', 'Helmer', 'Huntington', 'Varlotta', 'Bruno', 'Merriman Jr.']
17
['College of Wooster (left in 2015)', 'Oberlin College (left in 2016)', 'Antioch College', 'Wittenberg University (left in 2015)', 'University of Mount Union (left in 2015)', 'Kenyon College', 'Denison University', 'Ohio Northern University', 'Capital University (left in 2016)', 'Muskingum University (left in 2016)', 'Otterbein University', 'Ohio Wesleyan University', 'Baldwin Wallace University', 'Heidelberg University', 'Hiram College', 'Marietta College (left in 2016)', 'University of Mount Union (started in June 2015)']
17
['$911,651', '$829,913', '$507,672', '$463,504', '$453,800', '$451,698', '$435,322', '$414,716', '$388

In [31]:
data = {'First Name': firstname, 'Last Name': lastname, 
                  'College': college, 'Salary': salary}
df = pd.DataFrame(data=data)
print(df.shape)
print(df.isnull().sum())
df.head(17)

(17, 4)
College       0
First Name    0
Last Name     0
Salary        0
dtype: int64


,College,First Name,Last Name,Salary
0,College of Wooster (left in 2015),Grant,Cornwell,"$911,651"
1,Oberlin College (left in 2016),Marvin,Krislov,"$829,913"
2,Antioch College,Mark,Roosevelt,"$507,672"
3,Wittenberg University (left in 2015),Laurie,Joyner,"$463,504"
4,University of Mount Union (left in 2015),Richard,Giese,"$453,800"
5,Kenyon College,Sean,Decatur,"$451,698"
6,Denison University,Adam,Weinberg,"$435,322"
7,Ohio Northern University,Daniel,Dibiasio,"$414,716"
8,Capital University (left in 2016),Denvy,Bowman,"$388,570"
9,Muskingum University (left in 2016),Anne,Steele,"$384,233"


# 4) Stretch Goals

- Write a web scraper that can scrape "Data Scientist" job listings from indeed.com.
- Practice your regex skills: <https://regexone.com/>
- Look ahead to some of the topics from later this week:
 - Tokenization
 - Stemming
 - Lemmatization
 - Chunking
 - Part of Speech Tagging
 - Named Entity Recognition
 - Document Classification
 

In [32]:
address_list = ""

with open('data.txt', 'r', encoding='utf-8') as f:
  for cnt, line in enumerate(f):
    if cnt % 5 == 0: # 0 - Name, 1 - Phone, 2 - Address, 3- Email
      address_list += line
      
print(address_list)

Dave Martin
Charles Harris
Eric Williams
Corey Jefferson
Jennifer Martin-White
Erick Davis
Neil Patterson
Laura Jefferson
Maria Johnson
Michael Arnold
Michael Smith
Erik Stuart
Laura Martin
Barbara Martin
Linda Jackson
Eric Miller
Dave Arnold
Jennifer Jacobs
Neil Wilson
Kurt Jackson
Mary Jacobs
Michael White
Jennifer Jenkins
Sam Wright
John Davis
Eric Davis
Laura Jackson
John Williams
Michael Martin
Maggie Brown
Erik Wilson
Elizabeth Arnold
Jane Martin
Travis Johnson
Laura Jefferson
Tom Williams
Jennifer Taylor
Erick Wright
Steve Doe
Kurt Davis
Corey Harris
Nicole Taylor
Elizabeth Davis
Maggie Jenkins
Linda Davis
Dave Moore
Linda Jenkins
Eric White
Laura Robinson
Charles Patterson
Joe Jackson
Michael Johnson
Corey Miller
James Moore
Jennifer Stuart
Charles Martin
Eric Wilks
Elizabeth Arnold
John Miller
Corey Jackson
Sam Thomas
Patricia Thomas
Jennifer Davis
Patricia Brown
Barbara Williams
James Taylor
Eric Harris
Travis Anderson
Sam Robinson
Steve Robinson
Mary Wilson
Sam Wilson
Charle